In [1]:
#
# Update package history based on latest available statistics from ecosyste.ms and anaconda
#
# (C) Open Energy Transition (OET)
# Distributed via MIT license
#

import os.path
import json
import io
from datetime import datetime, timedelta

from IPython.display import HTML, display

try:
    import requests
except:
    !pip install request
    import requests

try:
    import pandas as pd
except:
    !pip install pandas
    import pandas as pd

try:
    import itables
    from itables import init_notebook_mode
    from itables import to_html_datatable
except:
    !pip install itables
    import itables
    from itables import init_notebook_mode
    from itables import to_html_datatable

try:
    import plotly.graph_objects as go
except:
    !pip install plotly
    import plotly.graph_objects as go

try:
    import numpy as np
except:
    !pip install numpy
    import numpy as np

try:
    import openpyxl
except:
    !pip install openpyxl
    import openpyxl

try:
    import pyarrow
except:
    !pip install pyarrow
    import pyarrow

try:
    from dash import Dash, dash_table, dcc, html, Input, Output, callback
except:
    !pip install dash
    from dash import Dash, dash_table, dcc, html, Input, Output, callback

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

init_notebook_mode(all_interactive=False)

# define necessary pathes towards ecosyste.ms
URL_projects = "https://ost.ecosyste.ms/api/v1/projects"
URL_packages = "https://ost.ecosyste.ms/api/v1/projects/packages"

FILE_TO_SAVE_AS = "ecosystems_packages.json" # the name you want to save file as

# define necessary path towards anaconda (latest available parquet file via S3
# s3_path = 's3://anaconda-package-data/conda/monthly/2025/2025-01.parquet'
input_dt = datetime(datetime.today().year, datetime.today().month, datetime.today().day)
first = input_dt.replace(day=1)
res = first - timedelta(days=1)
s3_path = 's3://anaconda-package-data/conda/monthly/'+res.date().strftime('%Y')+'/'+res.date().strftime('%Y-%m')+'.parquet'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [2]:
# define some required packages
def get_julia_pkg_downloads(package_name):
    """
    Fetch the monthly download statistics for a Julia package by name.

    Args:
        package_name (str): The name of the Julia package.

    Returns:
        dict: A dictionary containing download statistics or an error message.
    """
    try:
        # Define the URL for the Julia package statistics API
        url = f"https://juliapkgstats.com/api/v1/monthly_downloads/{package_name}"

        # Make the GET request
        response = requests.get(url)

        # Raise an exception for HTTP errors
        response.raise_for_status()

        # Parse the JSON response
        data = response.json()

        return {
            "package": package_name,
            "monthly_downloads": data.get("total_requests", "Data not available"),
            "details": data
        }

    except requests.exceptions.HTTPError as http_err:
        return {"error": f"HTTP error occurred: {http_err}"}
    except requests.exceptions.RequestException as req_err:
        return {"error": f"Request error occurred: {req_err}"}
    except Exception as e:
        return {"error": f"An error occurred: {e}"}

def get_ost_detail(OST_URL, FILE_TO_SAVE_AS):
    resp = requests.get(OST_URL) # requesting the projects from OST server

    with open(FILE_TO_SAVE_AS, "wb") as f: # opening a file handler to create new file
        f.write(resp.content) # writing content to file

    return (f)

def get_datatable(FILE_TO_READ_FROM):
    with open(FILE_TO_READ_FROM, 'r', encoding='utf-8') as f:
        content = f.read()
        f.close()

    return (content)

# Create a Plotly table
def plot_dataframe_as_table(df):
    """
    Create a Plotly table from a pandas DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame to visualize.

    Returns:
        Plotly figure
    """
    fig = go.Figure(data=[go.Table(
        header=dict(
            values=list(df.columns),
            fill_color='lightblue',
            align='center',
            font=dict(color='black', size=14)
        ),
        cells=dict(
            values=[df[col] for col in df.columns],
            fill_color='lightgrey',
            align='center',
            font=dict(color='black', size=12)
        )
    )])
    fig.update_layout(title="Usage of Open Source Projects in Energy Modelling - January 2025", margin=dict(l=0, r=0, t=30, b=0))
    fig.show()


In [3]:
if os.path.isfile(FILE_TO_SAVE_AS+'3.parque'):
    # read from downloaded files
    print ('Read the available local files ...')
    df_conda = pd.read_parquet(FILE_TO_SAVE_AS+'3.parque')

else:
    # download all data
    print ('Download the data from the individual servers ...')
    resp = get_ost_detail(URL_packages, FILE_TO_SAVE_AS+'1.json') # requests the packages from OST server
    resp = get_ost_detail(URL_projects, FILE_TO_SAVE_AS+'2.json') # requests the packages from OST server
    df_conda = pd.read_parquet(s3_path, engine='pyarrow') # Load the Parquet file into a DataFrame
    df_conda.to_parquet(FILE_TO_SAVE_AS+'3.parque')

df_packages = pd.read_json(io.StringIO(get_datatable(FILE_TO_SAVE_AS+'1.json')))
df_projects = pd.read_json(io.StringIO(get_datatable(FILE_TO_SAVE_AS+'2.json')))

print ('Necessary data now are available.')

Download the data from the individual servers ...
Necessary data now are available.


In [4]:
names = []
url = []
description = []
category = []
sub_category = []
language = []
license = []
download_counts = []
total_dependent_repos_count = []
stars = []
doi = []
citations = []
forks = []
contributors = []
develop_distr_score = []
share_top_contributor = []
past_year_issues_count = []
created = []
updated = []

In [5]:
# df_projects.columns
# df_packages.columns
# df_projects.iloc[5]['issues_stats']

In [6]:
for index, row in df_packages.iterrows():
    name = row['name']
    if row['packages'][0]['namespace'] != None:
        owner = row['packages'][0]['namespace'][11:]
        repro = row['packages'][0]['repository_url'][len(row['packages'][0]['namespace'])+len('https://')+1:]
    else:
        owner = ''
        repro = ''

    package_downloads = 0
    dependent_repos_count = 0
    for package_manager in range(len(row['packages'])):
        if row['packages'][package_manager]['downloads']:
                if row['packages'][package_manager]['downloads_period'] == "last-month":
                    package_downloads += row['packages'][package_manager]['downloads']

        if row['packages'][package_manager]['dependent_repos_count']:
                dependent_repos_count += row['packages'][package_manager]['dependent_repos_count']

    if row['language'] == "Python":
        conda_downloads = df_conda[df_conda['pkg_name'] == row['name']]['counts'].sum()
        package_downloads += conda_downloads

    try:
        download_counts.append(package_downloads)
    except:
        download_counts.append(0)

    try:
        contributors.append(row['repository']['commit_stats']['total_committers'])
    except:
        contributors.append(0)

    names.append(name)
    stars.append(row['repository']['stargazers_count'])
    past_year_issues_count.append(row['issues_stats']['past_year_issues_count'])
    url.append(row['url'])
    description.append(row['description'])
    category.append(row['category'])
    sub_category.append(row['sub_category'])
    language.append(row['language'])
    license.append(row['repository']['license'])
    citations.append(row['total_citations'])
    forks.append(row['repository']['forks_count'])
    created.append(datetime.strptime(row['repository']['created_at'], '%Y-%m-%dT%H:%M:%S.%fZ').strftime('%Y/%m'))
    updated.append(datetime.strptime(row['repository']['updated_at'], '%Y-%m-%dT%H:%M:%S.%fZ').strftime('%Y/%m'))
    total_dependent_repos_count.append(dependent_repos_count)
    develop_distr_score.append(row['commits']['dds'])

In [7]:
df_extract = pd.DataFrame()
df_extract['Project Name'] = names
df_extract['Project Name'] = names
df_extract['Category'] = category
df_extract['Sub Category'] = sub_category
df_extract['Created'] = created
df_extract['Updated'] = updated
df_extract['License'] = license
df_extract['Language'] = language
df_extract['Citations'] = citations
df_extract['Stars'] = stars
df_extract['Contributors'] = contributors
df_extract['DDS'] = develop_distr_score
df_extract['Forks'] = forks
df_extract['Dependents'] = total_dependent_repos_count
df_extract['PM Downloads'] = download_counts
df_extract['PY Issues']= past_year_issues_count

In [19]:
df_extract

,Project Name,Created,Updated,Citations,Stars,Contributors,DDS,Forks,Dependents,PM Downloads,PY Issues
0,pandapower,2017/01,2024/10,0,863,156,0.825620,481,75,30425,123
1,PyPSA,2016/01,2024/10,238,1239,92,0.808072,454,46,10209,96
2,PyPowSyBl,2020/11,2024/10,0,56,30,0.698709,12,1,7882,53
3,oemof-solph,2015/11,2024/10,175,302,72,0.708161,126,23,3019,54
4,openTEPES,2020/07,2024/10,7,39,8,0.417258,23,1,1129,9
5,FINE,2018/07,2025/01,0,73,48,0.790623,43,2,944,4
6,Minpower,2011/04,2024/04,0,71,1,0.000000,33,2,643,1
7,GridPath,2016/08,2024/10,0,95,12,0.214765,36,0,532,11
8,Calliope,2013/09,2024/10,123,299,22,0.392771,93,5,465,92
9,PowerSystems.jl,2017/12,2024/10,15,308,42,0.504858,79,0,182,137


In [9]:
# This project are lost between OpenSustain.tech and this script: OSeMOSYS, times_model, SpineOpt.jl
df_extract = df_extract[df_extract['Project Name'].isin(
    ['Antares Simulator', 'AnyMOD.jl', 'balmorel', 'Calliope', 'FINE', 'GenX', 'GridPath', 'NemoMod.jl',
     'Minpower', 'oemof-solph', 'openTEPES', 'OSeMOSYS', 'pandapower', 'powersimulationsdynamics.jl',
     'PowerSystems.jl',
     #'PowSyBl Open Load Flow',
     'PyPowSyBl', 'PyPSA', 'pypsa-earth', 'Sienna', 'SpineOpt.jl',
     'switch-model', 'Temoa', 'TulipaEnergyModel.jl', 'times_model'])]

# walk through Julia packages as typically they are named without the ".jl" at the end
for index, row in df_extract[df_extract['Language'] == 'Julia'].iterrows():
    try:
        if row['Project Name'][-3:] == '.jl':
            df_extract.loc[df_extract["Project Name"] == row['Project Name'], "PM Downloads"] = \
                int(get_julia_pkg_downloads(row['Project Name'][0:-3])['monthly_downloads'])
        else:
            df_extract.loc[df_extract["Project Name"] == row['Project Name'], "PM Downloads"] = \
                int(get_julia_pkg_downloads(row['Project Name'])['monthly_downloads'])
    except:
        df_extract.loc[df_extract["Project Name"] == row['Project Name'], "PM Downloads"] = 0

# df_extract

In [10]:
## The following projects can not be evaluated since two different API from ecosyste.ms are needed.
## This is simpilfied in future versions. For now the data is hardcoded here for the 17.01.2025
new_projects = pd.DataFrame([
    {   'Project Name': 'OSeMOSYS',
        'Stars': 163,
        'Citations': 568,
        'PM Downloads': 0,
        'Dependents': 0,
        'License': 'apache-2.0',
        'Language': 'Python',
        'PY Issues': 45,
        'Contributors': 11,
        'Forks': 104,
        'Created': '2016/10',
        'Updated': '2023/06',
        'DDS': 0,
    },
    {   'Project Name': 'times_model',
        'Stars': 116,
        'Citations': 0,
        'PM Downloads': 0,
        'Dependents': 0,
        'License': 'gpl-3.0',
        'Language': 'GAMS',
        'PY Issues': 0,
        'Contributors': 3,
        'Forks': 40,
        # creation not clearly indicated
        # guess based on https://iea-etsap.org/docs/Documentation_for_the_TIMES_Model-Part-I_July-2016.pdf
        'Created': '1990/07',
        'Updated': '2025/01',
        'DDS': 0,
    },
    {   'Project Name': 'Dispa-SET',
        'Stars': 86,
        'Citations': 0,
        'PM Downloads': 0,
        'Dependents': 0,
        'License': 'eupl-1.2',
        'Language': 'GAMS',
        'PY Issues': 16,
        'Contributors': 13,
        'Forks': 39,
        'Created': '2018/10',
        'Updated': '2024/04',
        'DDS': 0,
    }
])

# Append new data to the DataFrame
df_extract = pd.concat([df_extract, new_projects], ignore_index=True)

In [11]:
# how to enter data into the official repository?
df_extract.loc[df_extract.index[df_extract['Project Name'] == 'Antares Simulator'].values[0], 'License'] = 'mpl-2.0'
df_extract.loc[df_extract.index[df_extract['Project Name'] == 'FINE'].values[0], 'License'] = 'mit'
df_extract.loc[df_extract.index[df_extract['Project Name'] == 'Minpower'].values[0], 'License'] = 'mit'
df_extract.loc[df_extract.index[df_extract['Project Name'] == 'pandapower'].values[0], 'License'] = 'bsd-3-clause'
df_extract.loc[df_extract.index[df_extract['Project Name'] == 'switch-model'].values[0], 'License'] = 'apache-2.0'
df_extract.loc[df_extract.index[df_extract['Project Name'] == 'Temoa'].values[0], 'Language'] = 'Python'
df_extract.loc[df_extract.index[df_extract['Project Name'] == 'PyPowSyBl'].values[0], 'Language'] = 'Python'

df_extract.drop(columns=['Category', 'License', 'Sub Category', 'Owner', 'Repository', 'Language'], axis=1, errors='ignore', inplace=True)
df_extract.style.format({
    'Citations':'{:,.0f}',
    'Stars':'{:,.0f}',
    'DDS':'{:,.2f}',
    'PM Downloads':'{:,.0f}',
    'PY Issues':'{:,.0f}'
})
itables.show(
    # df_extract.loc[:, df_extract.columns != 'Repository'],
    df_extract,
    layout={"top1": "searchBuilder"},
    buttons=["copyHtml5", "csvHtml5", "excelHtml5"],
    searchBuilder={},
    lengthMenu=[25, 50],
    order=[[0, "asc"]]
)

# Remark:
#   DDS ... development distribution score (the smaller the number the better; but 0 means no data available)
#   PM .. previous month (0 means either no downloads or not tracked/shared from the repository owner)
#   PY .. previous year (0 means either no issues or not tracked/shared from the repository owner)

In [ ]:
# prepared to create the HTML table
#
# html = to_html_datatable(df_extract, display_logo_when_loading=False)
# with open('Blog #1 insights-energy-modelling.html', 'w', encoding='utf-8') as f:
#    f.write(html)
#
# try:
#     import gupload
#     from pydrive.auth import GoogleAuth
#     from google.colab import auth
# except:
#     !pip install gupload
#     !pip install pydrive
#     !pip install google
#     import gupload
#     from pydrive.auth import GoogleAuth
#     from google.colab import auth
#
# auth.authenticate_user() # Authenticate and create the PyDrive client.
# !gupload --to '1kded1hVoYRYyyhWibraw9DTW7cL3Qcfd' 'Blog #1 insights-energy-modelling.html'